# ㈱ストラクチャー配布のコード

以下で配布されている断面算定のコードを使用。
この章はコードのコピペになります。

- https://www.structure.jp/src_code.html


In [1]:
import math


In [2]:
class Steel:

    def __init__(self, size='', f=235, num_form=2):
        # size: 部材寸法の初期値
        # f: F値の初期値(N/mm2) または　'400N'(400N級) '490N'(490N級)
        # num_form: 出力時の小数以下の桁数(-1の場合は四捨五入しない)
        self.size_default = size
        self.f_default = 235
        if f == '400N':
            self.f_default = 235
        elif f == '490N':
            self.f_default = 325
        elif type(f) == int:
            self.f_default = f
        self.num_form = num_form
        # ここで使用している形状番号の定義
        self.shape_h = 1
        self.shape_wc = 2
        self.shape_lc = 3
        self.shape_box = 4
        self.shape_pipe = 5

    def calc_ft(self, size='', f=0):
        # 長期許容引張応力度と許容圧縮力の計算
        # f: F値(N/mm2) '400N'(400N級) '490N'(490N級) 省略時は初期値
        # 戻り値 (ft, nt) ft:長期許容引張応力度(N/mm2), nt:長期許容引張力(kN)
        (shape, section, values) = self.get_section(size)
        if values == None:
            return
        ft = self.get_f_value(f) / 1.5
        nt = 0.1 * ft * values[0]  # N/mm2 * cm2 -> kN
        return (self.out_form(ft), self.out_form(nt))

    def calc_fc(self, size='', lkx=0, lky=0, f=0):
        # 長期許容圧縮応力度と許容圧縮力の計算
        # lkx:強軸方向の座屈長(m) 省略時は座屈を考慮しない,
        # lky:弱軸方向の座屈長(m) 省略時はlkx, f:F値(N/mm2) 省略時は初期値
        # f: F値(N/mm2) '400N'(400N級) '490N'(490N級) 省略時は初期値
        # 戻り値 (fc, nc) fc:長期許容圧縮応力度(N/mm2), nc:長期許容圧縮力(kN)
        (shape, section, values) = self.get_section(size)
        if values == None:
            return
        if lky < 0.01:
            # lky が省略された場合は lky = lkx
            lky = lkx
        f_value = self.get_f_value(f)
        fc = f_value / 1.5
        if lkx > 0.01:
            # 座屈の考慮
            # values[] 0:A, 1:Ix, 2:Iy, 3:Zx, 4:Zy, 5:ix, 6:iy
            ix = values[5]
            iy = values[6]
            # 限界細長比 Λ (E = 205000)
            lam = math.sqrt(2023265.5 / (0.6 * f_value))
            lam1 = max(100 * lkx / ix, 100 * lky / iy)  # 細長比 λ
            lam2 = math.pow(lam1 / lam, 2)  # (λ / Λ) ^ 2
            if lam1 < lam:  # λ < Λ
                fc = f_value * (1.0 - 0.4 * lam2) / (1.5 + 0.667 * lam2)
            else:
                fc = 0.277 * f_value / lam2

        nc = 0.1 * fc * values[0]  # N/mm2 * cm2 -> kN
        return (self.out_form(fc), self.out_form(nc))

    def calc_fb(self, size='', lb=0, m2_m1=2, f=0):
        # 許容曲げ応力度と許容曲げの計算
        # lb:圧縮フランジの支点間距離(m), m2_m1:M2/M1の値
        # f: F値(N/mm2) '400N'(400N級) '490N'(490N級) 省略時は初期値
        # 戻り値 (fb, ma) fc:長期許容曲げ応力度(N/mm2), ma:長期許容曲げ(kN.m)
        (shape, section, values) = self.get_section(size)
        if values == None:
            return
        f_value = self.get_f_value(f)
        fb = f_value / 1.5
        if (shape < self.shape_box) and (lb > 0.01):
            # 横座屈を考慮する
            plb = 0.3  # pλb
            c = 1.0  # 曲げ勾配による補正係数
            if abs(m2_m1) < 1:
                plb = 0.6 + 0.3 * m2_m1
                c = 1.75 + 1.05 * m2_m1 + 0.3 * m2_m1 * m2_m1
                if c < 1.0:
                    c = 1.0
                elif c > 2.3:
                    c = 2.3
            e = 20500000  # ヤング係数 (N/cm2)
            g = 7900000  # せん断弾性係数 (N/cm2)
            # values[] 0:A, 1:Ix, 2:Iy, 3:Zx, 4:Zy, 5:ix, 6:iy
            iy = values[2]
            zx = values[3]
            # jw:サンブナン捩り定数, iw:曲げ捩り定数
            (jw, iw) = self.get_jw_iw(shape, section, values)
            # me:横座屈モーメント(N.cm)
            me = math.pow(math.pi / (100 * lb), 4) * e * iy * e * iw
            me += math.pow(math.pi / (100 * lb), 2) * e * iy * g * jw
            me = c * math.sqrt(me)
            lam = math.sqrt(100 * f_value * zx / me)  # λb
            elb = 1.291  # eλb
            nu = 1.5 + 0.667 * math.pow((lam / elb), 2)  # ν
            if lam < plb:  # λb < pλb
                fb = f_value / nu
            elif lam < elb:  # pλb < λb < eλb
                fb = (f_value / nu) * (1.0 - 0.4 * (lam - plb) / (elb - plb))
            else:  # eλb < λb
                fb = f_value / (lam * lam * 2.17)

        ma = 0.001 * fb * values[3]  # N/mm2 * cm3 -> kN.m
        return (self.out_form(fb), self.out_form(ma))

    def get_section(self, s):
        # 文字列 s から部材寸法と断面性能を取得
        # 戻り値(shape, section, values)
        # shape:部材種別(1:H, 2:WC, 3:LC, 4:BOX, 5:PIPE), section[]:部材寸法
        # values[]: 断面性能のリスト 0:A, 1:Ix, 2:Iy, 3:Zx, 4:Zy, 5:ix, 6:iy, 7:Cy
        section = []
        shape = 0  # 1:H, 2:WC, 3:LC, 4:BOX, 5:PIPE
        shape_name = ['H', 'WC', 'LC', 'BOX', 'PIPE']
        if s == '':
            s = self.size_default
        s = s.upper()
        for i in range(len(shape_name)):
            if s.find(shape_name[i]) != -1:
                shape = i + 1
                s = s.replace(shape_name[i], '')
        if shape > 0:
            s = s.replace('-', '')
            section = s.split('*')
            try:
                for i in range(len(section)):
                    section[i] = float(section[i])
            except:
                shape = 0

        values = []
        if shape == 0:
            print('断面寸法の指定に誤りがある')
        elif self.invalid_section(shape, section):
            print('断面の指定に誤りがある')
        elif shape == self.shape_h:
            values = self.h_section(section)
        elif shape == self.shape_wc:
            values = self.wc_section(section)
        elif shape == self.shape_lc:
            values = self.lc_section(section)
        elif shape == self.shape_box:
            values = self.box_section(section)
        else:
            values = self.pipe_section(section)

        return (shape, section, values)

    def invalid_section(self, shape, section):
        # 形鋼サイズの検証
        # 戻り値 1:エラーがある
        n = len(section)
        for i in range(n):
            if section[i] < 0.01:
                return 1
        if shape == self.shape_h:  # H
            if (n != 4) and (n != 5):
                return 1
            elif (section[0] < 2 * section[3]) or (section[1] < section[2]):
                return 1
        elif shape == self.shape_wc:  # WC
            if n != 4:
                return 1
            elif (section[0] < 2 * section[3]) or (section[1] < section[2]):
                return 1
        elif shape == self.shape_lc:  # LC
            if n != 4:
                return 1
            elif (section[0] < 2 * section[3]) or (section[1] < 2 * section[3]):
                return 1
        elif shape == self.shape_box:  # Box
            if (n != 2) and (n != 3):
                return 1
            elif section[0] < 2 * section[1]:
                return 1
        else:  # Pipe
            if n != 2:
                return 1
            elif section[0] < 2 * section[1]:
                return 1
        return 0

    def get_f_value(self, f):
        # F値の取得
        # 戻り値 F値
        if f == '400N':
            return 235
        elif f == '490N':
            return 325
        elif type(f) == int:
            if f == 0:
                return self.f_default
            else:
                return f

    def get_jw_iw(self, shape, section, values):
        # サンブナン捩り定数・曲げ捩り定数の計算
        # values: 断面性能のリスト 0:A, 1:Ix, 2:Iy, 3:Zx, 4:Zy, 5:ix, 6:iy, 7:Cy
        # 戻り値 (jw, iw) jw:サンブナン捩り定数, iw:曲げ捩り定数
        jw = 0
        iw = 0
        if shape == self.shape_h:
            b1 = 0.1 * section[1]  # mm -> cm
            t1 = 0.1 * section[2]
            t2 = 0.1 * section[3]
            h1 = 0.1 * section[0] - t2
            jw = (2 * b1 * (t2 ** 3) + h1 * (t1 ** 3)) / 3
            iw = values[2] * h1 * h1 / 4
        elif shape == self.shape_wc:
            t1 = 0.1 * section[2]  # mm -> cm
            t2 = 0.1 * section[3]
            b1 = 0.1 * section[1] - t1 / 2
            h1 = 0.1 * section[0] - t2
            jw = (2 * b1 * (t2 ** 3) + h1 * (t1 ** 3)) / 3
            c1 = values[7] - t1 / 2
            c2 = 1 - h1 * h1 * values[0] / (4 * values[1])
            iw = h1 * h1 * (values[2] + c1 * c1 * values[0] * c2) / 4
        elif shape == self.shape_lc:
            # section[]: 0:H, 1:B, 2:C, 3:t
            t = 0.1 * section[3]  # mm -> cm
            h = 0.1 * section[0] - t
            b = 0.1 * section[1] - t
            c = 0.1 * section[2] - t / 2
            c1 = values[7] - t / 2
            c2 = 1 - h * h * values[0] / (4 * values[1])
            jw = (2 * b + h + 2 * c) * (t ** 3) / 3
            iw = h * h * (values[2] + c1 * c1 * values[0] * c2) / 4
        return (jw, iw)

    def h_section(self, section):
        # H形鋼の断面性能
        # section[]: 0:H, 1:B, 2:tw, 3:tf, 4:r
        # 戻り値: 断面性能のリスト 0:A, 1:Ix, 2:Iy, 3:Zx, 4:Zy, 5:ix, 6:iy

        # jis[]: 登録済みの形鋼の寸法と断面性能(JISによる)
        # 0:H, 1:B, 2:tw, 3:tf, 4:r, 5:A, 6:Ix, 7:Iy, 8:Zx, 9:Zy, 10:ix, 11:iy
        jis = [[100, 50, 5, 7, 8, 11.9, 187, 14.8, 37.5, 5.91, 3.98, 1.12],
               [125, 60, 6, 8, 8, 16.7, 409, 29.1, 65.5, 9.71, 4.95, 1.32],
               [150, 75, 5, 7, 8, 17.8, 666, 49.5, 88.8, 13.2, 6.11, 1.66],
               [175, 90, 5, 8, 8, 22.9, 1210, 97.5, 138, 21.7, 7.26, 2.06],
               [198, 99, 4.5, 7, 8, 22.7, 1540, 113, 156, 22.9, 8.25, 2.24],
               [200, 100, 5.5, 8, 8, 26.7, 1810, 134, 181, 26.7, 8.23, 2.24],
               [248, 124, 5, 8, 8, 32.0, 3450, 255, 278, 41.1, 10.4, 2.82],
               [250, 125, 6, 9, 8, 37.0, 3960, 294, 317, 47.0, 10.4, 2.82],
               [298, 149, 5.5, 8, 13, 40.8, 6320, 442, 424, 59.3, 12.4, 3.29],
               [300, 150, 6.5, 9, 13, 46.8, 7210, 508, 481, 67.7, 12.4, 3.29],
               [346, 174, 6, 9, 13, 52.5, 11000, 791, 638, 91.0, 14.5, 3.88],
               [350, 175, 7, 11, 13, 62.9, 13500, 984, 771, 112, 14.6, 3.96],
               [396, 199, 7, 11, 13, 71.4, 19800, 1450, 999, 145, 16.6, 4.50],
               [400, 200, 8, 13, 13, 83.4, 23500, 1740, 1170, 174, 16.8, 4.56],
               [446, 199, 8, 12, 13, 83.0, 28100, 1580, 1260, 159, 18.4, 4.36],
               [450, 200, 9, 14, 13, 95.4, 32900, 1870, 1460, 187, 18.6, 4.43],
               [496, 199, 9, 14, 13, 99.3, 40800, 1840, 1650, 185, 20.3, 4.31],
               [500, 200, 10, 16, 13, 112, 46800, 2140, 1870, 214, 20.4, 4.36],
               [506, 201, 11, 19, 13, 129, 55500, 2580, 2190, 256, 20.7, 4.46],
               [596, 199, 10, 15, 13, 118, 66600, 1980, 2240, 199, 23.8, 4.10],
               [600, 200, 11, 17, 13, 132, 75600, 2270, 2520, 227, 24.0, 4.16],
               [606, 201, 11, 12, 20, 150, 88300, 2720, 2910, 270, 24.3, 4.26],
               [150, 100, 6, 9, 8, 26.4, 1000, 150, 135, 30.1, 6.17, 2.39],
               [194, 150, 6, 9, 8, 38.1, 2630, 507, 271, 67.6, 8.30, 3.65],
               [244, 175, 7, 11, 13, 55.5, 6040, 984, 495, 112, 10.4, 4.21],
               [294, 200, 8, 12, 13, 71.1, 11100, 1600, 756, 160, 12.5, 4.75],
               [340, 250, 9, 14, 13, 99.5, 21200, 3650, 1250, 292, 14.6, 6.05],
               [390, 300, 11, 18, 13, 133, 37900, 7200, 1940, 480, 16.9, 7.35],
               [440, 300, 11, 18, 13, 154, 54700, 8110, 2490, 540, 18.9, 7.26],
               [482, 300, 11, 15, 13, 141, 58300, 6760, 2420, 450, 20.3, 6.92],
               [488, 300, 11, 18, 13, 159, 68900, 8110, 2820, 540, 20.8, 7.14],
               [582, 300, 12, 17, 13, 169, 98900, 7660, 3400, 511, 24.2, 6.73],
               [588, 300, 12, 20, 13, 187, 114000, 9010, 3890, 601, 24.7, 6.94],
               [594, 302, 14, 23, 13, 217, 134000, 10600, 4500, 700, 24.8, 6.98],
               [692, 300, 13, 20, 18, 208, 168000, 9020, 4870, 601, 28.5, 6.59],
               [700, 300, 13, 24, 18, 232, 197000, 10800, 5640, 721, 29.2, 6.83],
               [792, 300, 14, 22, 18, 240, 248000, 9920, 6270, 661, 32.2, 6.44],
               [800, 300, 14, 26, 18, 264, 286000, 11700, 7160, 781, 33.0, 6.67],
               [890, 299, 15, 23, 18, 267, 339000, 10300, 7610, 687, 35.6, 6.20],
               [900, 300, 16, 28, 18, 306, 404000, 12600, 8990, 842, 36.4, 6.43],
               [912, 302, 18, 34, 18, 360, 491000, 15700, 10800, 1040, 36.9, 6.59],
               [100, 100, 6, 8, 8, 21.6, 378, 134, 75.6, 26.7, 4.18, 2.49],
               [125, 125, 6.5, 9, 8, 30.0, 839, 293, 134, 46.9, 5.29, 3.13],
               [150, 150, 7, 10, 8, 39.7, 1620, 563, 216, 75.1, 6.40, 3.77],
               [175, 175, 7.5, 11, 13, 51.4, 2900, 984, 331, 112, 7.50, 4.37],
               [200, 200, 8, 12, 13, 63.5, 4720, 1600, 472, 160, 8.62, 5.02],
               [200, 204, 12, 12, 13, 71.5, 4980, 1700, 498, 167, 8.35, 4.88],
               [250, 250, 9, 14, 13, 91.4, 10700, 3650, 860, 292, 10.8, 6.32],
               [250, 255, 14, 14, 13, 104, 11400, 3880, 912, 304, 10.5, 6.11],
               [294, 302, 12, 12, 13, 106, 16600, 5510, 1130, 365, 12.5, 7.20],
               [300, 300, 10, 15, 13, 118, 20200, 6750, 1350, 450, 13.1, 7.55],
               [300, 305, 15, 15, 13, 133, 21300, 7100, 1420, 466, 12.6, 7.3],
               [344, 348, 10, 16, 13, 144, 32800, 11200, 1910, 646, 15.1, 8.84],
               [350, 350, 12, 19, 13, 172, 39800, 13600, 2280, 776, 15.2, 8.89],
               [388, 402, 15, 15, 22, 179, 49000, 16300, 2520, 809, 16.6, 9.55],
               [394, 398, 11, 18, 22, 187, 56100, 18900, 2850, 951, 17.3, 10.1],
               [400, 400, 13, 21, 22, 219, 66600, 22400, 3330, 1120, 17.5, 10.1],
               [400, 408, 21, 21, 22, 251, 70900, 23800, 3540, 1170, 16.8, 9.75],
               [414, 405, 18, 28, 22, 295, 92800, 31000, 4480, 1530, 17.7, 10.2],
               [428, 407, 20, 35, 22, 361, 119000, 39400, 5570, 1930, 18.2, 10.4],
               [458, 417, 30, 50, 22, 529, 187000, 60500, 8170, 2900, 18.8, 10.7],
               [498, 432, 45, 70, 22, 770, 298000, 94400, 12000, 4370, 19.7, 11.1]]

        for i in range(len(jis)):
            if section[:4] == jis[i][:4]:
                return jis[i][5:]
        # 登録されていない場合は計算値
        return self.h_values(section)

    def h_values(self, section):
        # H形鋼の断面性能の計算
        # 戻り値: 断面性能のリスト 0:A, 1:Ix, 2:Iy, 3:Zx, 4:Zy, 5:ix, 6:iy
        res = [0 for i in range(7)]
        h = 0.1 * section[0]  # mm -> cm
        b = 0.1 * section[1]
        tw = 0.1 * section[2]
        tf = 0.1 * section[3]
        h1 = h - 2 * tf
        res[0] = b * tf * 2 + h1 * tw  # A
        res[1] = b * h * h * h / 12 - (b - tw) * h1 * h1 * h1 / 12  # Ix
        res[2] = tf * b * b * b / 6 + h1 * tw * tw * tw / 12  # Iy
        if len(section) > 4:
            r1 = 0.1 * section[4]
            res[0] += 0.8584 * r1 * r1  # A
            h2 = 0.5 * h - tf - 0.2234 * r1
            res[1] += 0.8584 * r1 * r1 * h2 * h2  # Ix
            h3 = 0.2234 * r1 + 0.5 * tw
            res[2] += 0.8584 * r1 * r1 * h3 * h3  # Iy
        # Zx
        res[3] = res[1] / (0.5 * h)
        # Zy
        res[4] = res[2] / (0.5 * b)
        # ix
        res[5] = math.sqrt(res[1] / res[0])
        # iy
        res[6] = math.sqrt(res[2] / res[0])
        return res

    def wc_section(self, section):
        # 溝形綱の断面性能
        # section[]: 0:H, 1:B, 2:tw, 3:tf
        # 戻り値: 断面性能のリスト 0:A, 1:Ix, 2:Iy, 3:Zx, 4:Zy, 5:ix, 6:iy, 7:Cy

        # jis[]: 登録済みの形鋼の寸法と断面性能(JISによる)
        # 0:H, 1:B, 2:tw, 3:tf, 4:A, 5:Ix, 6:Iy, 7:Zx, 8:Zy, 9:ix, 10:iy, 11:Cy
        jis = [[75, 40, 5, 7, 8.82, 75.3, 12.2, 20.1, 4.47, 2.92, 1.17, 1.28],
               [100, 50, 5, 7.5, 11.9, 188, 26.0, 37.6, 7.52, 3.97, 1.48, 1.54],
               [125, 65, 6, 8, 17.1, 424, 61.8, 67.8, 13.4, 4.98, 1.90, 1.90],
               [150, 75, 6.5, 10, 23.7, 861, 117, 115, 22.4, 6.03, 2.22, 2.28],
               [150, 75, 9, 12.5, 30.6, 1050, 147, 140, 28.3, 5.86, 2.19, 2.31],
               [180, 75, 7, 10.5, 27.2, 1380, 131, 153, 24.3, 7.12, 2.19, 2.13],
               [200, 80, 7.5, 11, 31.3, 1950, 168, 195, 29.1, 7.88, 2.32, 2.21],
               [200, 90, 8, 13.5, 38.7, 2490, 277, 249, 44.2, 8.02, 2.68, 2.74],
               [250, 90, 9, 13, 44.1, 4180, 294, 334, 44.5, 9.74, 2.58, 2.40],
               [250, 90, 11, 14.5, 51.2, 4680, 329, 374, 49.9, 9.56, 2.54, 2.40],
               [300, 90, 9, 13, 48.6, 6440, 309, 429, 45.7, 11.5, 2.52, 2.22],
               [300, 90, 10, 15.5, 55.7, 7410, 360, 494, 54.1, 11.5, 2.54, 2.34],
               [300, 90, 12, 16, 61.9, 7870, 379, 525, 56.4, 11.3, 2.48, 2.28],
               [380, 100, 10.5, 16, 69.4, 14500, 535, 763, 70.5, 14.5, 2.78, 2.41],
               [380, 100, 13, 16.5, 79.0, 15600, 565, 823, 73.6, 14.1, 2.67, 2.33],
               [380, 100, 13, 20, 85.7, 17600, 655, 926, 87.8, 14.3, 2.80, 2.54]]

        for i in range(len(jis)):
            if section[:4] == jis[i][:4]:
                return jis[i][4:]

        print('未登録の溝形綱断面が使用されている')

    def lc_section(self, section):
        # リップ溝形綱の断面性能
        # section[]: 0:H, 1:B, 2:C, 3:t
        # 戻り値: 断面性能のリスト 0:A, 1:Ix, 2:Iy, 3:Zx, 4:Zy, 5:ix, 6:iy, 7:Cy

        # jis[]: 登録済みの形鋼の寸法と断面性能(JISによる)
        # 0:H, 1:B, 2:C, 3:t, 4:A, 5:Ix, 6:Iy, 7:Zx, 8:Zy, 9:ix, 10:iy, 11:Cy
        jis = [[60, 30, 10, 1.6, 2.072, 11.6, 2.56, 3.88, 1.32, 2.37, 1.11, 1.06],
               [60, 30, 10, 2.0, 2.537, 14.0, 3.01, 4.65, 1.55, 2.35, 1.09, 1.06],
               [60, 30, 10, 2.3, 2.872, 15.6, 3.32, 5.20, 1.71, 2.33, 1.07, 1.06],
               [70, 40, 25, 1.6, 3.032, 22.0, 8.00, 6.29, 3.64, 2.69, 1.62, 1.80],
               [75, 45, 15, 1.6, 2.952, 27.1, 8.71, 7.29, 3.13, 3.03, 1.72, 1.72],
               [75, 45, 15, 2.0, 3.637, 33.0, 10.5, 8.79, 3.76, 3.01, 1.70, 1.72],
               [75, 45, 15, 2.3, 4.137, 37.1, 11.8, 9.90, 4.24, 3.00, 1.69, 1.72],
               [90, 45, 20, 1.6, 3.352, 42.6, 10.5, 9.46, 5.86, 3.56, 1.77, 1.73],
               [90, 45, 20, 2.3, 4.712, 58.6, 14.2, 13.0, 5.14, 3.53, 1.74, 1.73],
               [90, 45, 20, 3.2, 6.367, 76.9, 18.3, 17.1, 6.57, 3.48, 1.69, 1.72],
               [100, 50, 20, 1.6, 3.672, 58.4, 14.0, 11.7, 4.47, 3.99, 1.95, 1.87],
               [100, 50, 20, 2.0, 4.537, 71.4, 16.9, 14.3, 5.40, 3.97, 1.93, 1.86],
               [100, 50, 20, 2.3, 5.172, 80.7, 19.0, 16.1, 6.06, 3.95, 1.92, 1.86],
               [100, 50, 20, 2.8, 6.205, 99.8, 20.0, 16.1, 7.44, 3.96, 1.91, 1.88],
               [100, 50, 20, 3.2, 7.007, 107, 24.5, 21.3, 7.81, 3.90, 1.87, 1.86],
               [100, 50, 20, 4.0, 8.548, 127, 28.7, 25.4, 9.13, 3.85, 1.83, 1.86],
               [100, 50, 20, 4.5, 9.469, 139, 30.9, 27.7, 9.82, 3.82, 1.81, 1.86],
               [120, 40, 20, 3.2, 7.007, 144, 15.3, 24.0, 5.71, 4.53, 1.48, 1.32],
               [120, 60, 20, 2.3, 6.092, 140, 31.3, 23.3, 8.10, 4.79, 2.27, 2.13],
               [120, 60, 20, 3.2, 8.287, 186, 40.9, 31.0, 10.5, 4.74, 2.22, 2.12],
               [120, 60, 25, 4.5, 11.72, 252, 58.0, 41.9, 15.5, 4.63, 2.22, 2.25],
               [125, 50, 20, 2.3, 5.747, 137, 20.6, 21.9, 6.22, 4.88, 1.89, 1.69],
               [125, 50, 20, 3.2, 7.807, 181, 26.6, 29.0, 8.02, 4.82, 1.85, 1.68],
               [125, 50, 20, 4.0, 9.548, 217, 33.1, 34.7, 9.38, 4.77, 1.81, 1.68],
               [125, 50, 20, 4.5, 10.59, 238, 33.5, 38.0, 10.0, 4.74, 1.78, 1.68],
               [150, 50, 20, 2.3, 6.322, 210, 21.9, 28.0, 6.33, 5.77, 1.86, 1.55],
               [150, 50, 20, 3.2, 8.607, 280, 28.3, 37.4, 8.19, 5.71, 1.81, 1.54],
               [150, 50, 20, 4.5, 11.72, 378, 35.7, 49.0, 10.5, 5.60, 1.75, 1.54],
               [150, 65, 20, 2.3, 7.012, 248, 41.1, 33.0, 9.37, 5.94, 2.42, 2.12],
               [150, 65, 20, 3.2, 9.567, 332, 53.8, 44.0, 12.2, 5.89, 2.37, 2.11],
               [150, 65, 20, 4.0, 11.75, 401, 63.7, 53.5, 14.5, 5.84, 2.33, 2.11],
               [150, 75, 20, 3.2, 10.21, 366, 76.4, 48.9, 15.3, 5.99, 2.74, 2.51],
               [150, 75, 20, 4.0, 12.55, 445, 91.0, 59.3, 18.2, 5.95, 2.69, 2.51],
               [150, 75, 20, 4.5, 13.97, 489, 99.2, 65.2, 19.8, 5.92, 2.66, 2.50],
               [150, 75, 25, 3.2, 10.53, 375, 83.6, 50.0, 17.3, 5.97, 2.82, 2.66],
               [150, 75, 25, 4.0, 12.95, 455, 99.8, 60.6, 20.6, 5.93, 2.78, 2.65],
               [150, 75, 25, 4.5, 14.42, 501, 109, 66.9, 22.5, 5.90, 2.75, 2.65],
               [200, 75, 20, 3.2, 11.81, 716, 84.1, 71.6, 15.8, 7.79, 2.67, 2.19],
               [200, 75, 20, 4.0, 14.55, 871, 100, 87.1, 18.9, 7.74, 2.62, 2.19],
               [200, 75, 20, 4.5, 16.22, 963, 109, 96.3, 20.6, 7.71, 2.60, 2.19],
               [200, 75, 25, 3.2, 12.13, 736, 92.3, 73.6, 17.8, 7.70, 2.76, 2.33],
               [200, 75, 25, 4.0, 14.95, 895, 110, 89.5, 21.3, 7.74, 2.72, 2.32],
               [200, 75, 25, 4.5, 16.67, 990, 121, 99.0, 23.3, 7.61, 2.69, 2.32],
               [250, 75, 25, 4.5, 18.92, 1690, 129, 135, 23.8, 9.44, 2.62, 2.07]]

        for i in range(len(jis)):
            if section[:4] == jis[i][:4]:
                return jis[i][4:]

        print('未登録のリップ溝形綱断面が使用されている')

    def box_section(self, section):
        # 正方形鋼管の断面性能
        # section[]: 0:B, 1:t, 2:r
        # 戻り値: 断面性能のリスト 0:A, 1:Ix, 2:Iy, 3:Zx, 4:Zy, 5:ix, 6:iy
        res = [0 for i in range(7)]
        d = 0.1 * section[0]  # mm -> cm
        t = 0.1 * section[1]
        d1 = d - 2 * t
        if len(section) < 3:
            res[0] = d * d - d1 * d1  # A
            res[1] = (d ** 4 - d1 ** 4) / 12  # Ix, Iy
        else:
            r = 0.1 * section[2]
            res[0] = 2 * (2 * d - 4 * r) * t + \
                (2 * r * t - t * t) * math.pi  # A
            r1 = r - t
            d2 = d - 2 * r
            c1 = (4 * (r * r + r1 * r + r1 * r1)) / \
                (3 * math.pi * (r1 + r)) + d2 / 2  # Ix, Iy
            res[1] = (d2 * (d ** 3 - d1 ** 3) + 2 * t * (d2 ** 3)) / 12
            res[1] += math.pi * (r * r - r1 * r1) * c1 * c1

        res[2] = res[1]
        res[3] = res[1] / (0.5 * d)
        res[4] = res[3]
        res[5] = math.sqrt(res[1] / res[0])
        res[6] = res[5]
        return res

    def pipe_section(self, section):
        # 円形鋼管の断面性能
        # section[]: 0:B, 1:t
        # 戻り値: 断面性能のリスト 0:A, 1:Ix, 2:Iy, 3:Zx, 4:Zy, 5:ix, 6:iy
        res = [0 for i in range(7)]
        t = 0.1 * section[1]  # mm -> cm
        r1 = 0.1 * section[0]
        r2 = r1 - 2 * t
        res[0] = (r1 * r1 - r2 * r2) * math.pi / 4
        res[1] = (r1 ** 4 - r2 ** 4) * math.pi / 64
        res[2] = res[1]
        res[3] = res[1] / (0.5 * r1)
        res[4] = res[3]
        res[5] = math.sqrt(res[1] / res[0])
        res[6] = res[5]
        return res

    def out_form(self, val):
        # 値 val を出力用の値に整形
        if self.num_form < 0:
            return val
        else:
            return round((10 ** self.num_form) * val) / (10 ** self.num_form)


## 動作確認用


In [3]:
obj = Steel('H-300*150*6.5*9')
(fb, ma) = obj.calc_fb(lb=3)
print('fb = ' + str(fb) + ', Ma = ' + str(ma))


fb = 109.73, Ma = 52.78


# Hops の実装


## Hops の初期設定


In [4]:
!pip install ghhops-server flask-ngrok


  Using cached ghhops_server-1.5.2-py2.py3-none-any.whl (10 kB)
  Using cached flask_ngrok-0.0.25-py3-none-any.whl (3.1 kB)
  Using cached Flask-2.1.2-py3-none-any.whl (95 kB)
  Using cached Werkzeug-2.1.2-py3-none-any.whl (224 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached importlib_metadata-4.11.4-py3-none-any.whl (18 kB)
  Using cached itsdangerous-2.1.2-py3-none-any.whl (15 kB)
  Using cached zipp-3.8.0-py3-none-any.whl (5.4 kB)
  Using cached MarkupSafe-2.1.1-cp39-cp39-win_amd64.whl (17 kB)


You should consider upgrading via the 'C:\Users\hiroa\Desktop\DigitalFUTURES\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
!pip install matplotlib numpy seaborn


  Using cached matplotlib-3.5.2-cp39-cp39-win_amd64.whl (7.2 MB)
  Using cached numpy-1.22.4-cp39-cp39-win_amd64.whl (14.7 MB)
  Using cached seaborn-0.11.2-py3-none-any.whl (292 kB)
  Using cached Pillow-9.1.1-cp39-cp39-win_amd64.whl (3.3 MB)
  Using cached kiwisolver-1.4.3-cp39-cp39-win_amd64.whl (55 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.33.3-py3-none-any.whl (930 kB)
  Using cached pandas-1.4.2-cp39-cp39-win_amd64.whl (10.5 MB)
  Using cached scipy-1.8.1-cp39-cp39-win_amd64.whl (36.9 MB)
  Using cached pytz-2022.1-py2.py3-none-any.whl (503 kB)


You should consider upgrading via the 'C:\Users\hiroa\Desktop\DigitalFUTURES\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [6]:
import rhino3dm
import seaborn as sns
import numpy
import ghhops_server as hs
from flask_ngrok import run_with_ngrok
from flask import Flask
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()


[DEBUG] Loaded backend module://matplotlib_inline.backend_inline version unknown.
[DEBUG] Loaded backend module://matplotlib_inline.backend_inline version unknown.


In [7]:
app = Flask(__name__)
run_with_ngrok(app)
hops = hs.Hops(app)


## コンポーネントの実装


In [8]:
from ghhops_server.params import HopsNumber
from ghhops_server.params import HopsString


@hops.component(
    "/moment",
    name="moment",
    inputs=[
        hs.HopsString("Name", "SN", "Section name like 'H-300*150*6.5*9'"),
        hs.HopsNumber("lb", "lb", "lb"),
        hs.HopsNumber("m2_m1", "m2_m1", "m2_m1"),
    ],
    outputs=[
        hs.HopsNumber("fb", "fb", "Long term fb N/mm2"),
        hs.HopsNumber("ma", "ma", "Long term ma kNm"),
    ]
)
def text(section_name, lb, m2_m1):
    obj = Steel(section_name)
    (fb, ma) = obj.calc_fb(lb=lb, m2_m1=m2_m1)
    return fb, ma


## サーバーの実行


In [10]:
app.run()


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


[INFO]  * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)


[DEBUG] Starting new HTTP connection (1): localhost:4040
[DEBUG] http://localhost:4040 "GET /api/tunnels HTTP/1.1" 200 839


 * Running on http://7382-240b-10-2161-6900-fd50-55e7-7958-c12d.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
